# Урок 8. Снижение размерности данных 

**1. Обучить любую модель классификации на датасете IRIS до применения PCA и после него. Сравнить качество классификации по отложенной выборке.**

In [1]:
import numpy as np

import matplotlib.pyplot as plt

# from sklearn import datasets
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import r2_score

**Классификация до применния PCA**

In [2]:
# X, y = datasets.load_iris(return_X_y=True)
X, y = make_classification(
    n_samples=100, n_features = 20, n_informative=5, n_classes=3, n_clusters_per_class=1, random_state = 42
)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [4]:
model = RandomForestClassifier(random_state = 42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [5]:
r2_score(y_test, y_pred)

0.5872641509433963

**Классификация после применния PCA**

In [6]:
def scale(X):
    X_ = X.astype(float)
    rows, cols = X_.shape
    # центрирование - вычитание из каждого значения среднего по строке
    means = X_.mean(0)
    for i in range(rows):
        for j in range(cols):
            X_[i, j] -= means[j]
    # деление каждого значения на стандартное отклонение
    std = np.std(X_, axis=0)
    for i in range(cols):
        for j in range(rows):
            X_[j][i] /= std[i]
            
    return X_

In [7]:
def pca(X_, min_var=5.0):
    X = scale(X_)
    # Найдем собственные векторы и собственные значения
    covariance_matrix = X.T.dot(X)
    eig_values, eig_vectors = np.linalg.eig(covariance_matrix)
    # сформируем список кортежей (собственное значение, собственный вектор)
    eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]
    # и отсортируем список по убыванию собственных значений
    eig_pairs.sort(key=lambda x: x[0], reverse=True)
    
    eig_sum = sum(eig_values)
    var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
    print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')
    # Сформируем вектор весов из собственных векторов, соответствующих главным компонентам
    n = len(eig_values)
    vectors = list()
    for i, var in enumerate(var_exp):
        if (var > min_var):
            vectors.append(eig_pairs[i][1].reshape(n,1))
    W = np.hstack(tuple(vectors))
    
    return X.dot(W)

In [8]:
Z = pca(X, min_var=3.0)
# print(f'W \n{W}')

Доля дисперсии, описываемая каждой из компонент 
[13.812199086013925, 12.199143237119674, 7.700224491872194, 7.108175376457125, 6.472666654406621, 6.353668923409872, 5.87303159177406, 5.608021539663833, 5.464156398782314, 4.525878490925064, 4.39588561333654, 4.122037807971374, 3.793479063389583, 3.2733423937898958, 3.043194075820405, 2.78008737489931, 2.024662420654246, 1.450145459713974, 5.016100672588775e-16, -1.5737589187121325e-15]


In [9]:
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(Z, y, test_size = 0.25, random_state = 42)

In [10]:
model.fit(X_train_pca, y_train_pca)
y_pred_pca = model.predict(X_test_pca)

In [11]:
r2_score(y_test_pca, y_pred_pca)

0.7051886792452831

**Вывод:** в данном случае при малом количестве объектов и пропорционально небольшом количестве информативных признаков метод главных компонент улучшил качество модели.

**2. Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()**

In [12]:
def pca_svd(X, min_vector=5.0):
    U, s, vh = np.linalg.svd(X)

    vectors = list()
    for i, eig_vector in enumerate(s):
        if (eig_vector > min_vector):
            vectors.append(eig_vector)
    
    n = len(vectors)
   
    Z = np.dot(U[:, :n], np.dot(np.diag(s[:n]), vh[:n, :]))
    
    return Z

In [13]:
Z_svd = pca_svd(X, min_vector=5.0)

In [14]:
X_train_pca_svd, X_test_pca_svd, y_train_pca_svd, y_test_pca_svd = train_test_split(Z, y, test_size = 0.25, random_state = 42)

In [15]:
model.fit(X_train_pca_svd, y_train_pca_svd)
y_pred_pca_svd = model.predict(X_test_pca_svd)

In [16]:
r2_score(y_test_pca_svd, y_pred_pca_svd)

0.7051886792452831

**Вывод:** результат совпадает с предыдущей реализацией, значит функция написана верно.